# Spaceship Titanic with DL
### 배경
* 우주의 미스터리를 풀기 위해 데이터 과학 기술이 필요한 2912년에 오신 것을 환영합니다. 4광년 떨어진 곳에서 전송을 받았는데 상태가 좋지 않습니다.

* 우주선 타이타닉은 한 달 전에 발사된 성간 여객선이었습니다. 약 13,000명의 승객을 태운 이 선박은 우리 태양계에서 가까운 별을 도는 새로 거주 가능한 세 개의 외계 행성으로 이민자들을 수송하는 첫 항해를 시작했습니다.

* 첫 번째 목적지인 55 Cancri E로 가는 도중 Alpha Centauri를 돌던 중 부주의한 우주선 Titanic이 먼지 구름 속에 숨겨진 시공간 변칙과 충돌했습니다. 안타깝게도 1000년 전의 이름과 비슷한 운명을 맞이했습니다. 배는 온전했지만 승객의 거의 절반이 다른 차원으로 이동했습니다!

### 데이터 정보
* *PassengerId*
    - 각 승객의 고유 ID. 각 Id는 승객이 함께 여행하는 그룹을 나타내고 그룹 내의 번호를 나타내는 형식을 취합니다 . 그룹의 사람들은 종종 가족 구성원이지만 항상 그런 것은 아닙니다.

* *HomePlanet*
    - 승객이 출발한 행성으로, 일반적으로 승객이 거주하는 행성입니다.

* *CryoSleep*
    - 승객이 항해 기간 동안 냉동 수면 선택했는지 여부를 나타냅니다. cryosleep의 승객은 객실에 갇혀 있습니다.

* *Cabin*
    - 승객이 머무르는 캐빈 번호. 형식을 취합니다 deck/num/side. 여기 에서 Port 또는 Starboard 가 side될 수 있습니다.

* *Destination*
    - 승객이 내릴 행성.

* *Age*
    - 승객의 나이.

* *VIP*
    - 승객이 항해 중 특별 VIP 서비스 비용을 지불했는지 여부.

* *RoomService, FoodCourt, ShoppingMall, Spa, VRDeck*
    - 승객이 Spaceship Titanic 의 다양한 고급 편의 시설 각각에 대해 청구한 금액입니다.

* *Name*
    - 승객의 성과 이름.

* *Transported*
    - 승객이 다른 차원으로 이동했는지 여부. 정답 데이터입니다.

## import library

In [29]:
import pandas as pd
import tensorflow as tf

In [30]:
# from google.colab import drive
# drive.mount('/content/drive')

## Data Load
### Read CSV files wit pandas

In [31]:
train_data = pd.read_csv("spaceship_titanic_train_data.csv")
train_labels = pd.read_csv("spaceship_titanic_train_labels.csv")

test_data = pd.read_csv("spaceship_titanic_test_data.csv")
test_labels = pd.read_csv("spaceship_titanic_test_labels.csv")

train = pd.concat([train_data, train_labels], axis=1)
test = pd.concat([test_data, test_labels], axis=1)

### Preprocessing
* 결측치 제거 후 데이터 로더에 연결

In [32]:
train = train.fillna(method='bfill')
test = test.fillna(method='bfill')

F:\Soma\temp\ipykernel_14248\1631429711.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train = train.fillna(method='bfill')
F:\Soma\temp\ipykernel_14248\1631429711.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test = test.fillna(method='bfill')


In [33]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep          bool
Cabin            object
Destination      object
Age             float64
VIP                bool
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [34]:
# 일부 dtype은 tensor로 변경 불가
train['HomePlanet'] = train['HomePlanet'].astype('category')
train['CryoSleep'] = train['CryoSleep'].map({True: 1, False: 0})
train['VIP'] = train['VIP'].astype('string')
train['Transported'] = train['Transported'].astype('int')

test['HomePlanet'] = test['HomePlanet'].astype('category')
test['CryoSleep'] = test['CryoSleep'].map({True: 1, False: 0})
test['VIP'] = test['VIP'].astype('string')
test['Transported'] = test['Transported'].astype('int')

In [35]:
train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,2513_01,Earth,0,F/575/P,TRAPPIST-1e,28.0,False,0.0,55.0,0.0,656.0,0.0,Loree Mathison,0
1,2774_02,Earth,0,F/575/P,TRAPPIST-1e,17.0,False,0.0,1195.0,31.0,0.0,0.0,Crisey Mcbriddley,0
2,8862_04,Europa,1,C/329/S,55 Cancri e,28.0,False,0.0,0.0,0.0,0.0,0.0,Alramix Myling,1
3,8736_02,Mars,0,F/1800/P,TRAPPIST-1e,20.0,False,0.0,2.0,289.0,976.0,0.0,Tros Pota,1
4,0539_02,Europa,1,C/18/P,55 Cancri e,36.0,False,0.0,0.0,0.0,0.0,0.0,Achyon Nalanet,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6949,6076_01,Earth,0,G/988/S,TRAPPIST-1e,18.0,False,14.0,2.0,144.0,610.0,0.0,Therry Cames,1
6950,5537_01,Mars,0,F/1063/S,TRAPPIST-1e,50.0,False,690.0,0.0,30.0,762.0,428.0,Herms Bancy,0
6951,5756_06,Earth,0,F/1194/P,PSO J318.5-22,22.0,False,158.0,0.0,476.0,0.0,26.0,Karena Briggston,0
6952,0925_01,Mars,0,F/191/P,TRAPPIST-1e,34.0,False,379.0,0.0,1626.0,0.0,0.0,Skix Kraie,0


In [36]:
test

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0337_02,Mars,0,F/63/S,TRAPPIST-1e,19.0,False,417.0,349.0,634.0,3.0,1057.0,Weros Perle,1
1,2891_01,Earth,0,G/460/S,TRAPPIST-1e,18.0,False,4.0,904.0,0.0,0.0,1.0,Gleney Ortinericey,0
2,8998_01,Earth,1,G/1449/S,TRAPPIST-1e,41.0,False,0.0,0.0,0.0,0.0,0.0,Gerry Englence,0
3,1771_01,Earth,0,G/291/P,TRAPPIST-1e,35.0,False,0.0,338.0,436.0,0.0,0.0,Antone Cardner,1
4,9034_02,Europa,1,D/288/P,TRAPPIST-1e,43.0,False,0.0,0.0,0.0,0.0,0.0,Errairk Crakete,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1734,7656_01,Earth,1,G/1244/S,TRAPPIST-1e,16.0,False,0.0,0.0,0.0,0.0,0.0,Moniey Belley,0
1735,3437_02,Earth,1,G/553/S,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Carly Pager,1
1736,1384_01,Earth,0,E/105/S,TRAPPIST-1e,17.0,False,21.0,0.0,690.0,260.0,5.0,Violan Mayods,0
1737,6300_01,Mars,1,F/1303/P,TRAPPIST-1e,42.0,False,0.0,0.0,0.0,0.0,0.0,Risps Pure,1


### Data Loader

In [37]:
batch_size = 32

def df_to_dataset(dataframe, label_name="Transported", shuffle=True, batch_size=batch_size):
    dataframe = dataframe.copy()
    labels = dataframe.pop(label_name)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.repeat()
    ds = ds.batch(batch_size)

    return ds


In [38]:
train_ds = df_to_dataset(train)
train_ds

<BatchDataset element_spec=({'PassengerId': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'HomePlanet': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'CryoSleep': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Cabin': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Destination': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Age': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'VIP': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'RoomService': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'FoodCourt': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'ShoppingMall': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Spa': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'VRDeck': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Name': TensorSpec(shape=(None,), dtype=tf.string, name=None)}, TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [39]:
test_ds = df_to_dataset(test, shuffle=False)
test_ds

<BatchDataset element_spec=({'PassengerId': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'HomePlanet': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'CryoSleep': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Cabin': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Destination': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Age': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'VIP': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'RoomService': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'FoodCourt': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'ShoppingMall': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Spa': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'VRDeck': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Name': TensorSpec(shape=(None,), dtype=tf.string, name=None)}, TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [40]:
# for t, l in train_ds:
#   print(t, l)
#   break

for t, l in test_ds:
  print(t, l)
  break


{'PassengerId': <tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'0337_02', b'2891_01', b'8998_01', b'1771_01', b'9034_02',
       b'3706_02', b'3614_01', b'3476_04', b'1210_01', b'1289_01',
       b'1555_01', b'2861_01', b'0618_01', b'1406_01', b'5188_01',
       b'6511_03', b'0680_01', b'9132_02', b'8098_04', b'1399_01',
       b'8306_03', b'6482_02', b'8429_01', b'8865_01', b'3564_01',
       b'0237_01', b'8078_01', b'3533_01', b'0011_01', b'9103_01',
       b'5303_02', b'3959_01'], dtype=object)>, 'HomePlanet': <tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'Mars', b'Earth', b'Earth', b'Earth', b'Europa', b'Mars',
       b'Earth', b'Earth', b'Earth', b'Earth', b'Europa', b'Earth',
       b'Earth', b'Mars', b'Earth', b'Earth', b'Earth', b'Mars',
       b'Europa', b'Earth', b'Earth', b'Mars', b'Earth', b'Earth',
       b'Earth', b'Europa', b'Earth', b'Mars', b'Earth', b'Earth',
       b'Earth', b'Earth'], dtype=object)>, 'CryoSleep': <tf.Tensor: shape=(32,), dtype=int64, 

In [41]:
list(set(train['VIP']))

['True', 'False']

## Preprocessing with layers

In [42]:
inputs = {
  'CryoSleep': tf.keras.Input(shape=(), dtype='int64'),
  'HomePlanet': tf.keras.Input(shape=(), dtype='string'),
  'RoomService': tf.keras.Input(shape=(), dtype='float64'),
  'FoodCourt': tf.keras.Input(shape=(), dtype='float64'),
  'ShoppingMall': tf.keras.Input(shape=(), dtype='float64'),
  'Spa': tf.keras.Input(shape=(), dtype='float64'),
  'VRDeck': tf.keras.Input(shape=(), dtype='float64'),
  'VIP': tf.keras.Input(shape=(), dtype='string'),
  'Cabin': tf.keras.Input(shape=(), dtype='string'),
  'Destination':tf.keras.Input(shape=(), dtype='string'),
  'Age': tf.keras.Input(shape=(), dtype='float64')
}

split_text = tf.strings.split(inputs['Cabin'], sep="/")

# Convert index to one-hot; e.g. [2] -> [0,1].
type_output = tf.keras.layers.CategoryEncoding(num_tokens=2, output_mode='one_hot')(inputs['CryoSleep'])
print(type_output.shape)
dense_type = tf.keras.layers.Dense(2,kernel_initializer = 'he_normal')(type_output)
dense_type = tf.keras.layers.BatchNormalization()(dense_type)
dense_type =tf.keras.layers.Activation('relu')(dense_type)
print(dense_type.shape)

vip = tf.keras.layers.StringLookup(vocabulary=["True", "False"], num_oov_indices=0, output_mode='one_hot')(inputs['VIP'])
print(vip.shape)

# Convert size strings to indices; e.g. ['small'] -> [1].
size_output = tf.keras.layers.StringLookup(vocabulary=list(set(train['HomePlanet'])))(inputs['HomePlanet'])
size_output = tf.keras.layers.Reshape([-1])(size_output)
size_output = tf.keras.layers.Lambda(lambda x: tf.cast(x, dtype=tf.float64))(size_output)
# print(size_output.shape)
dense_size = tf.keras.layers.Dense(2,kernel_initializer = 'he_normal')(size_output)
dense_size =tf.keras.layers.BatchNormalization()(size_output)
dense_size =tf.keras.layers.Activation('relu')(size_output)
# print(dense_size.shape)

# Normalize the numeric inputs; e.g. [2.0] -> [0.0].
weight_output = tf.keras.layers.Normalization(
      axis=None, mean=2.0, variance=1.0)(inputs['RoomService'])
weight_output = tf.keras.layers.Reshape([-1])(weight_output)
print(weight_output.shape)
dense_weight = tf.keras.layers.Dense(2,kernel_initializer = 'he_normal')(weight_output)
dense_weight =tf.keras.layers.BatchNormalization()(weight_output)
dense_weight =tf.keras.layers.Activation('relu')(weight_output)
#foodCourt
food_weight_output = tf.keras.layers.Normalization(
      axis=None, mean=2.0, variance=1.0)(inputs['FoodCourt'])
food_weight_output = tf.keras.layers.Reshape([-1])(food_weight_output)
print(food_weight_output.shape)
food_weight_output = tf.keras.layers.Dense(2,kernel_initializer = 'he_normal')(food_weight_output)
food_weight_output = tf.keras.layers.BatchNormalization()(food_weight_output)
food_weight_output = tf.keras.layers.Activation('relu')(food_weight_output)
# ShoppingMall
shopping_output = tf.keras.layers.Normalization(
      axis=None, mean=2.0, variance=1.0)(inputs['ShoppingMall'])
shopping_output = tf.keras.layers.Reshape([-1])(shopping_output)
print(food_weight_output.shape)
shopping_output = tf.keras.layers.Dense(2,kernel_initializer = 'he_normal')(shopping_output)
shopping_output = tf.keras.layers.BatchNormalization()(shopping_output)
shopping_output = tf.keras.layers.Activation('relu')(shopping_output)
#Spa
sha_output = tf.keras.layers.Normalization(
      axis=None, mean=2.0, variance=1.0)(inputs['Spa'])
sha_output = tf.keras.layers.Reshape([-1])(sha_output)
print(food_weight_output.shape)
sha_output = tf.keras.layers.Dense(2,kernel_initializer = 'he_normal')(sha_output)
sha_output = tf.keras.layers.BatchNormalization()(sha_output)
sha_output = tf.keras.layers.Activation('relu')(sha_output)
#VRDeck
vr_output = tf.keras.layers.Normalization(
      axis=None, mean=2.0, variance=1.0)(inputs['VRDeck'])
vr_output = tf.keras.layers.Reshape([-1])(vr_output)
print(food_weight_output.shape)
vr_output = tf.keras.layers.Dense(2,kernel_initializer = 'he_normal')(vr_output)
vr_output = tf.keras.layers.BatchNormalization()(vr_output)
vr_output = tf.keras.layers.Activation('relu')(vr_output)

# 캐빈 데이터를 분할합니다
cabin_split = tf.strings.split(inputs['Cabin'], "/")

# 마지막 요소만 선택합니다
cabin_first = cabin_split.to_tensor()[:, 0]
cabin_last = cabin_split.to_tensor()[:, -1]

# StringLookup 레이어를 사용하여 one-hot 인코딩을 수행합니다
# cabin_first = tf.keras.layers.StringLookup(vocabulary=["C","D","E","F","G"], num_oov_indices=1, output_mode='one_hot')(cabin_first)
# cabin_first = tf.keras.layers.Dense(2,kernel_initializer = 'he_normal')(cabin_first)
# cabin_first = tf.keras.layers.BatchNormalization()(cabin_first)
# cabin_first = tf.keras.layers.Activation('relu')(cabin_first)

cabin_output = tf.keras.layers.StringLookup(vocabulary=["S", "P"], num_oov_indices=1, output_mode='one_hot')(cabin_last)
cabin_output = tf.keras.layers.Dense(2,kernel_initializer = 'he_normal')(cabin_output)
cabin_output = tf.keras.layers.BatchNormalization()(cabin_output)
cabin_output = tf.keras.layers.Activation('relu')(cabin_output)

# list(set(train['HomePlanet']))
string_lookup_layer = tf.keras.layers.StringLookup(vocabulary=list(set(train['HomePlanet'])),num_oov_indices=0,output_mode='one_hot')(inputs['HomePlanet'])
string_lookup_layer = tf.keras.layers.Dense(2,kernel_initializer = 'he_normal')(string_lookup_layer)
string_lookup_layer = tf.keras.layers.BatchNormalization()(string_lookup_layer)
string_lookup_layer = tf.keras.layers.Activation('relu')(string_lookup_layer)

string_destination = tf.keras.layers.StringLookup(vocabulary=list(set(train['Destination'])),num_oov_indices=0,output_mode='one_hot')(inputs['Destination'])
string_destination = tf.keras.layers.Dense(2,kernel_initializer = 'he_normal')(string_destination)
string_destination = tf.keras.layers.BatchNormalization()(string_destination)
string_destination = tf.keras.layers.Activation('relu')(string_destination)

#age
age_output = tf.keras.layers.Normalization(axis=None, mean=2.0, variance=1.0)(inputs['Age'])
age_output = tf.keras.layers.Reshape([-1])(age_output)
print(age_output.shape)
age_output = tf.keras.layers.Dense(2,kernel_initializer = 'he_normal')(age_output)
age_output =tf.keras.layers.BatchNormalization()(age_output)
age_output =tf.keras.layers.Activation('relu')(age_output)


# x = tf.concat([dense_type, dense_size, dense_weight, dense_room_service], -1)# batch, 특징 (여기로 합쳐라)
x = tf.concat([dense_type, dense_size,age_output, dense_weight,food_weight_output,shopping_output,sha_output,vr_output,cabin_output,string_lookup_layer,string_destination], -1)# batch, 특징 (여기로 합쳐라)
# x = tf.concat([dense_type, dense_size,age_output, dense_weight,food_weight_output,shopping_output,sha_output,vr_output,cabin_output,cabin_first,string_lookup_layer,string_destination], -1)# batch, 특징 (여기로 합쳐라)
x = tf.keras.layers.Dense(64)(x)
x = tf.keras.layers.BatchNormalization()(x)
x =tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dense(4)(x)
x = tf.keras.layers.BatchNormalization()(x)
x =tf.keras.layers.Activation('relu')(x)

outputs = tf.keras.layers.Dense(1)(x) # Sigmoid, BCE loss

# outputs = {
#   'CryoSleep': type_output,
#   'HomePlanet': size_output,
#   'RoomService': weight_output,
#   'VIP': vip,
#   's': split_text
# }

preprocessing_model = tf.keras.Model(inputs, outputs)

(None, 2)
(None, 2)
(None, 2)
(None, 1)
(None, 1)
(None, 2)
(None, 2)
(None, 2)
(None, 1)


## Input and preprocessing Layers

### String Look Up


In [43]:
#  tf.strings.split(train['Cabin'], "/")[:,-1:].numpy()

In [44]:
# inputs = {
#     'Cabin': tf.keras.Input(shape=(), dtype='string')
# }

# # 캐빈 데이터를 분할합니다
# cabin_split = tf.strings.split(inputs['Cabin'], "/")

# # 마지막 요소만 선택합니다
# cabin_last = cabin_split.to_tensor()[:, -1]

# # StringLookup 레이어를 사용하여 one-hot 인코딩을 수행합니다
# cabin_output = tf.keras.layers.StringLookup(vocabulary=["S", "P"], num_oov_indices=1, output_mode='one_hot')(cabin_last)

# outputs = {
#     'Cabin': cabin_output
# }

# model = tf.keras.Model(inputs, outputs)

In [45]:
# for t, l in train_ds:
#     pred = model(t)
#     print(pred['Cabin'])
#     break

In [46]:
# # s = "F/575/P"
# tf.strings.split(s, sep="/")

In [47]:
list(set(train['HomePlanet']))

['Europa', 'Mars', 'Earth']

In [48]:
list(set(train['Destination']))

['55 Cancri e', 'PSO J318.5-22', 'TRAPPIST-1e']

In [49]:
# string_lookup_layer = tf.keras.layers.StringLookup(
#     vocabulary=list(set(train['HomePlanet'])),
#     num_oov_indices=0,
#     output_mode='one_hot')
# """
# int 0, 1, 2 (idx)
# one_hot [1, 0, 0]
# multi_hot [1, 0, 1] (다중입력)
# """

# string_lookup_layer(list(train['HomePlanet'].to_numpy()))

### Category Encoding

In [50]:
# train['CryoSleep']

In [51]:
# one_hot_layer = tf.keras.layers.CategoryEncoding(
#     num_tokens=2, output_mode='one_hot')
# one_hot_layer(list(train['CryoSleep'].to_numpy()))

### Normalization

In [52]:
normalization_layer = tf.keras.layers.Normalization(mean=1.0, variance=10.0)

# 각 입력 값에서 mean 값을 뺍니다. 예를 들어, 입력이 [1., 2., 3.]라면, 결과는 [-2., -1., 0.]이 됩니다.
# 다음으로, 이 결과를 variance의 제곱근 값, 즉 sqrt(2.)로 나눕니다. 따라서 결과는 [-2/sqrt(2), -1/sqrt(2), 0]가 됩니다. (정규화 과정)

print(normalization_layer(train['RoomService'][:5]).numpy())
print(train['RoomService'][:5].to_numpy())

[-0.31622776 -0.31622776 -0.31622776 -0.31622776 -0.31622776]
[0. 0. 0. 0. 0.]


### Model Train

In [53]:
import os
from os.path import isdir, join
use_colab = True
assert use_colab in [True, False]
if use_colab:
    checkpoint_dir ='./drive/MyDrive/train_ckpt/spectrogram/exp1'
    if not os.path.isdir(checkpoint_dir):
        os.makedirs(checkpoint_dir)
else:
    checkpoint_dir = 'spectrogram/exp1'

In [54]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir,
                                                 save_weights_only=True,
                                                 monitor='val_loss',
                                                 mode='auto',
                                                 save_best_only=True,
                                                 verbose=1)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20,
                                                     monitor='val_loss',
                                                     restore_best_weights=True,

                                                     verbose=1)
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss',  # 관찰할 지표
                              factor=0.2,  # 학습률을 줄이는 비율
                              patience=4,  # 몇 번의 에포크 동안 감소하지 않아야 하는지
                              min_lr=1e-9)

In [55]:
preprocessing_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [56]:
max_epochs = 150

history = preprocessing_model.fit(train_ds,
                                  epochs=max_epochs,
                                  steps_per_epoch=len(train) // batch_size,
                                  callbacks=[cp_callback,early_stopping_cb,reduce_lr],
                                  validation_data=test_ds,
                                  validation_steps=len(test) // batch_size)

Epoch 1/150


F:\Anaconda3_envs\tensor2\lib\site-packages\keras\engine\functional.py:637: UserWarning: Input dict contained keys ['PassengerId', 'Name'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


215/217 [============================>.] - ETA: 0s - loss: 0.6516 - accuracy: 0.4955
Epoch 1: val_loss improved from inf to 0.64554, saving model to ./drive/MyDrive/train_ckpt/spectrogram\exp1
217/217 [==============================] - 7s 21ms/step - loss: 0.6514 - accuracy: 0.4967 - val_loss: 0.6455 - val_accuracy: 0.4942 - lr: 0.0010
Epoch 2/150
214/217 [============================>.] - ETA: 0s - loss: 0.5769 - accuracy: 0.6396
Epoch 2: val_loss did not improve from 0.64554
217/217 [==============================] - 4s 17ms/step - loss: 0.5769 - accuracy: 0.6404 - val_loss: 0.6718 - val_accuracy: 0.4942 - lr: 0.0010
Epoch 3/150
214/217 [============================>.] - ETA: 0s - loss: 0.5056 - accuracy: 0.7452
Epoch 3: val_loss improved from 0.64554 to 0.62602, saving model to ./drive/MyDrive/train_ckpt/spectrogram\exp1
217/217 [==============================] - 4s 18ms/step - loss: 0.5058 - accuracy: 0.7447 - val_loss: 0.6260 - val_accuracy: 0.4965 - lr: 0.0010
Epoch 4/150
215/217